In [40]:
def stream_file(path):
    with open(path, 'r') as csv:
        next(csv) # file本身就是generator
        for line in csv:
            text, label = line[:-3], int(line[-2]) # 最后一个是换行符
            yield text, label

In [41]:
sf = stream_file('../data/csvfinal.csv')

In [22]:
text, label = next(sf)
print(text)

手机 便宜   但 声音 有点 小,0

手机 便宜   但 声音 有点 小


In [42]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        pass
    return docs, y

In [43]:
# load stop_words
stopwords = open('../data/stop_words.txt', 'r', encoding='GBK').read() #打不开则用GBK编码， 默认使用utf8
stops = stopwords.splitlines() # list

In [44]:
count = 0
def tokenizer(text):
    try:
        return text.split() # 以任意空格分割句子
    except Exception as e:
        print(e)
        print(text)
        global count
        count += 1
        return []

In [45]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
import numpy as np
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None, 
                         tokenizer=tokenizer, stop_words=stops)
clf = SGDClassifier(loss='log', random_state=1, n_iter=1, class_weight={0:18, 1:1}) # 基于随机梯度下降的logistic回归模型
classes = np.array([0, 1])
for _ in range(32): # 分批次，每次选取10000个进行训练
    X_train, y_train = get_minibatch(sf, size=10000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)

In [46]:
X_test, y_test = get_minibatch(sf, size=100000)
X_test = vect.transform(X_test)
print("accuracy:{}".format(clf.score(X_test, y_test))) # 精确度与线性模型不相上下，但速度大大提升

accuracy:0.84319


In [47]:
clf.partial_fit(X_test, y_test)

SGDClassifier(alpha=0.0001, average=False, class_weight={0: 18, 1: 1},
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=1,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=1, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [48]:
import pickle
pickle.dump(clf, open('sgdmodel0.85.pkl', 'wb'), protocol=4)

In [53]:
with open('sgdmodel0.85.pkl', 'rb') as f:
    clf = pickle.load(f)
label = {0:'negative', 1:'positive'}
example = ['手机 性能 很 差']
X = vect.transform(example)
print("label:%s\npossibility:%.2f%%" % (clf.predict(X)[0], np.max(clf.predict_proba(X))*100))

label:0
possibility:84.67%
